In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import VimeoVideo
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.utils.validation import check_is_fitted

This lesson is part of the DS Lab core curriculum. For that reason, this notebook can only be used on your WQU virtual machine.
That's why i didn't upload the dataset but the task followed by its answer down

In this project, you're working for a client who wants to create a model that can predict the price of apartments in the city of Buenos Aires — with a focus on apartments that cost less than $400,000 USD.WQU WorldQuant University Applied Data Science Lab QQQQ

# Prepare Data

Task 2.1.1: Write a function named wrangle that takes a file path as an argument and returns a DataFrame.

What's a function?
Write a function in Python.

In [5]:
def wrangle(filepath):
    #Read CSV into DataFrames
    df=pd.read_csv(filepath)
    return df

Task 2.1.2: Use your wrangle function to create a DataFrame df from the CSV file data/buenos-aires-real-estate-1.csv.

In [ ]:
df = wrangle("data/buenos-aires-real-estate-1.csv")
print("df shape:", df.shape)
df.head()

In [ ]:
At this point, 
your DataFrame df should have no more than 8,606 observations.

In [ ]:
# Check your work
assert (
    len(df) <= 8606
), f"`df` should have no more than 8606 observations, not {len(df)}."

For this project, we want to build a model for apartments in Buenos Aires proper ("Capital Federal") that cost less than $400,000. Looking at the first five rows of our DataFrame, we can already see that there properties that fall outside those parameters. So our first cleaning task is to remove those observations from our dataset. Since we're using a function to import and clean our data, we'll need to make changes there.

In [ ]:
def wrangle(filepath):
    #Read CSV into DataFrames
    df=pd.read_csv(filepath)
    #subset Data to Captial Federal property
    mask_ba=df['place_with_parent_names'].str.contains('Capital Federal')
    #subset Data to apartment property
    mask_apt=df['property_type']=='apartment'
    #subset data to price < 400,000 property
    mask_price=df['price_aprox_usd']<400_000
    #mask_price.head()
    df=df[mask_ba &  mask_apt & mask_price]
    return df

In [ ]:
# Check your work
assert (
    len(df) <= 1781
), f"`df` should have no more than 1781 observations, not {len(df)}."

# Explore Data Analysis

Task 2.1.4: Create a histogram of "surface_covered_in_m2". Make sure that the x-axis has the label "Area [sq meters]" and the plot has the title "Distribution of Apartment Sizes".

In [ ]:
plt.hist('surface_covered_in_m2')
plt.xlabel('Area [sq meters]')
plt.title('Distribution of Apartment Sizes')

Yikes! When you see a histogram like the one above, it suggests that there are outliers in your dataset. This can affect model performance — especially in the sorts of linear models we'll learn about in this project. To confirm, let's look at the summary statistics for the "surface_covered_in_m2" feature.

In [ ]:
df.describe()['surface_covered_in_m2']

Task 2.1.6: Add to your wrangle function so that it removes observations that are outliers in the "surface_covered_in_m2" column. Specifically, all observations should fall between the 0.1 and 0.9 quantiles for "surface_covered_in_m2".

In [ ]:
low,high=df['surface_covered_in_m2'].quantile([0.1,0.9])
mask_area=df['surface_covered_in_m2'].between(low,high)
mask_area.head()
#correct the wrangle function with the new mask that removing the outliers

Task 2.1.7: Create a scatter plot that shows price ("price_aprox_usd") vs area ("surface_covered_in_m2") in our dataset. Make sure to label your x-axis "Area [sq meters]" and your y-axis "Price [USD]"

In [ ]:
plt.scatter(x=df['surface_covered_in_m2'],y=df['price_aprox_usd'])
plt.xlabel('Area [sq meters]')
plt.ylabel('Price [USD]')

# Split

Task 2.1.8: Create the feature matrix named X_train, which you'll use to train your model. It should contain one feature only: ["surface_covered_in_m2"]. Remember that your feature matrix should always be two-dimensional.

In [ ]:
features = ["surface_covered_in_m2"]
X_train = df[features]
X_train.head()

In [ ]:
# Check your work
assert X_train.shape == (
    1343,
    1,
), f"The shape of `X_train` should be (1343, 1), not {X_train.shape}."

Task 2.1.9: Create the target vector named y_train, which you'll use to train your model. Your target should be "price_aprox_usd". Remember that, in most cases, your target vector should be one-dimensional.

In [ ]:
target = "price_aprox_usd"
y_train = df[target]
y_train.shape


In [ ]:
# Check your work
assert y_train.shape == (1343,)

# Build Model

**Baseline**

The first step in building a model is baselining. To do this, ask yourself how you will know if the model you build is performing well?" One way to think about this is to see how a "dumb" model would perform on the same data. Some people also call this a naïve or baseline model, but it's always a model makes only one prediction — in this case, it predicts the same price regardless of an apartment's size. So let's start by figuring out what our baseline model's prediction should be.

Task 2.1.10: Calculate the mean of your target vector y_train and assign it to the variable y_mean.

In [ ]:
y_mean = y_train.mean()
y_mean

Task 2.1.11: Create a list named y_pred_baseline that contains the value of y_mean repeated so that it's the same length at y

In [ ]:
y_pred_baseline = [y_mean]* len(y_train)
y_pred_baseline[:5]

Task 2.1.12: Add a line to the plot below that shows the relationship between the observations X_train and our dumb model's predictions y_pred_baseline. Be sure that the line color is orange, and that it has the label "Baseline Model".

In [ ]:
#use X_train.values that convert dataframe into array or
#use X_train['Surface...etc'] specify ur column to convert it into
#pandas series
plt.plot(X_train.values,y_pred_baseline,color='orange',label="Baseline Model")
plt.scatter(X_train, y_train)
plt.xlabel("Area [sq meters]")
plt.ylabel("Price [USD]")
plt.title("Buenos Aires: Price vs. Area")
plt.legend();

Looking at this visualization, it seems like our baseline model doesn't really follow the trend in the data. But, as a data scientist, you can't depend only on a subjective plot to evaluate a model. You need an exact, mathematically calculate performance metric. There are lots of performance metrics, but the one we'll use here is the mean absolute error.

Task 2.1.13: Calculate the baseline mean absolute error for your predictions in y_pred_baseline as compared to the true targets in y.

In [ ]:
mae_baseline = mean_absolute_error(y_train,y_pred_baseline)

print("Mean apt price", round(y_mean, 2))
print("Baseline MAE:", round(mae_baseline, 2))

# Iterate

The next step in building a model is iterating. This involves building a model, training it, evaluating it, and then repeating the process until you're happy with the model's performance. Even though the model we're building is linear, the iteration process rarely follows a straight line. Be prepared for trying new things, hitting dead-ends, and waiting around while your computer does long computations to train your model. ☕️ Let's get started!

The first thing we need to do is create our model — in this case, one that uses linear regression.

Task 2.1.14: Instantiate a LinearRegression model named model.

In [ ]:
model = LinearRegression()

In [ ]:
# Check your work
assert isinstance(model, LinearRegression)

Task 2.1.15: Fit your model to the data, X_train and y_train.

Fit a model to training data in scikit-learn.

In [ ]:
model.fit(X_train,y_train)

In [ ]:
# Check your work
check_is_fitted(model)

# Evaluate

The final step is to evaluate our model. In order to do that, we'll start by seeing how well it performs when making predictions for data that it saw during training. So let's have it predict the price for the houses in our training set

Task 2.1.16: Using your model's predict method, create a list of predictions for the observations in your feature matrix X_train. Name this array y_pred_training.

In [ ]:
y_pred_training = model.predict(X_train)
y_pred_training[:5]

In [ ]:
# Check your work
assert (
    len(y_pred_training) == 1343
), f"There should be 1343 predictions in `y_pred_training`, not {len(y_pred_training)}."

Task 2.1.17: Calculate your training mean absolute error for your predictions in y_pred_training as compared to the true targets in y_train

In [ ]:
mae_training = mean_absolute_error(y_train,y_pred_training)
print("Training MAE:", round(mae_training, 2))

Training MAE: 31248.26
Good news: Our model beat the baseline by over $10,000! That's a good indicator that it will be helpful in predicting apartment prices. But the real test is how the model performs on data that it hasn't seen before, data that we call the test set. In the future, you'll create your own test set before you train your model, but here we'll use one that's pre-made, and we'll evaluate the model using the WQU auto-grader.

Task 2.1.19: Extract the intercept from your model, and assign it to the variable intercept.

In [ ]:
intercept = model.intercept_
print("Model Intercept:", intercept)
assert any([isinstance(intercept, int), isinstance(intercept, float)])

Task 2.1.20: Extract the coefficient associated "surface_covered_in_m2" in your model, and assign it to the variable coefficient.

In [ ]:
coefficient = round(model.coef_[0],2)
print('Model coefficient for "surface_covered_in_m2":', coefficient)
assert any([isinstance(coefficient, int), isinstance(coefficient, float)])

Now that we have our intercept and coefficient, we need to insert them into a string that we can print out the complete equation

Task 2.1.21: Complete the code below and run the cell to print the equation that your model has determined for predicting apartment price based on size.

In [ ]:
print(f"apt_price={intercept}+{coefficient}*surface_covered")

Equation might work for some technical audiences, but visualization or generally much more effective communication tool — especially for non-technical audiences. So let's use the scatter plot we made at the beginning of this lesson and plot the line that that are equation would make.

Task 2.1.22: Add a line to the plot below that shows the relationship between the observations in X_train and your model's predictions y_pred_training. Be sure that the line color is red, and that it has the label "Linear Model".

In [ ]:
plt.plot(X_train.values,model.predict(X_train),color='red',label="Linear Model")
plt.scatter(X_train, y_train)
plt.xlabel("surface covered [sq meters]")
plt.ylabel("price [usd]")
plt.legend();